In [2]:
from collections import OrderedDict
from robustness import datasets
from robustness.tools.breeds_helpers import make_entity30
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.utils.data import sampler
import os
import torchvision
import pandas as pd
# for plotting
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

from trainer_functions.customImageNettrainer import build_resnet50, evaluate_customImageNet, train_customImageNet

In [3]:
data_dir = '/Users/rada/.mxnet/datasets/imagenet'
info_dir = '/Users/rada/Documents/GitHub/BREEDS-Benchmarks/imagenet_class_hierarchy/modified'

n_subset = 1500
lrs = [0.0005]

ret = make_entity30(info_dir, split="rand")
superclasses, subclass_split, label_map = ret
train_subclasses, test_subclasses = subclass_split

dataset_source = datasets.CustomImageNet(data_dir, train_subclasses)
loaders_source = dataset_source.make_loaders(workers=10, batch_size=64, shuffle_train=64, shuffle_val=64, subset=n_subset)
train_loader_source, val_loader_source = loaders_source

dataset_target = datasets.CustomImageNet(data_dir, test_subclasses)
loaders_target = dataset_target.make_loaders(workers=10, batch_size=64, shuffle_train=True, shuffle_val=True, subset=n_subset)
train_loader_target, val_loader_target = loaders_target


==> Preparing dataset custom_imagenet..
==> Preparing dataset custom_imagenet..


In [3]:
df = pd.DataFrame(columns=['accuracy', "lr", "state", "n_subset"])

for lr in lrs:
    device = torch.device("mps")
    tune_net = build_resnet50(device)

    optimizer = optim.Adam(tune_net.fc.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
    acc = train_customImageNet(tune_net, train_loader_source, val_loader_source, optimizer, scheduler, device=device, epochs=3)
    
    optimizer = optim.Adam(tune_net.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
    acc = train_customImageNet(tune_net, train_loader_source, val_loader_source, optimizer, scheduler, device=device, epochs=2)

    # the first layer is fine-tuned
    optimizer = optim.Adam(tune_net.layer1.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
    acc = train_customImageNet(tune_net, train_loader_target, val_loader_target, optimizer, scheduler, device=device, epochs=15)
    
    # adding results to dataframe
    dict = {}
    dict['accuracy'] = acc
    dict['lr'] = lr
    dict['state'] = "first layer"  # change when run
    dict['n_subset'] = n_subset
    df_temp = pd.DataFrame(dict, index=[0])
    df = pd.concat([df, df_temp])

# change when run
df.to_csv("Entity30_layer1.csv")

Building model...
Epoch 0, loss = 6.0252, lr 0.00050000
Got 12 / 6000 correct (0.20)

Epoch 1, loss = 4.6965, lr 0.00050000
Got 89 / 6000 correct (1.48)

Epoch 2, loss = 3.9834, lr 0.00050000
Got 280 / 6000 correct (4.67)

Epoch 0, loss = 1.3843, lr 0.00050000
Got 4050 / 6000 correct (67.50)

Epoch 1, loss = 0.9626, lr 0.00050000
Got 4484 / 6000 correct (74.73)

Epoch 0, loss = 1.8521, lr 0.00050000
Got 3391 / 6000 correct (56.52)

Epoch 1, loss = 1.2522, lr 0.00050000
Got 3415 / 6000 correct (56.92)

Epoch 2, loss = 1.7978, lr 0.00050000
Got 3451 / 6000 correct (57.52)

Epoch 3, loss = 1.7309, lr 0.00050000
Got 3481 / 6000 correct (58.02)

Epoch 4, loss = 1.7580, lr 0.00050000
Got 3468 / 6000 correct (57.80)

Epoch 5, loss = 1.2372, lr 0.00050000
Got 3470 / 6000 correct (57.83)

Epoch 6, loss = 1.1958, lr 0.00050000
Got 3492 / 6000 correct (58.20)

Epoch 7, loss = 1.6706, lr 0.00050000
Got 3506 / 6000 correct (58.43)

Epoch 8, loss = 2.0486, lr 0.00050000
Got 3531 / 6000 correct (58.8

In [4]:
df = pd.DataFrame(columns=['accuracy', "lr", "state", "n_subset"])

for lr in lrs:
    device = torch.device("mps")
    tune_net = build_resnet50(device)

    optimizer = optim.Adam(tune_net.fc.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
    acc = train_customImageNet(tune_net, train_loader_source, val_loader_source, optimizer, scheduler, device=device, epochs=3)
    
    optimizer = optim.Adam(tune_net.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
    acc = train_customImageNet(tune_net, train_loader_source, val_loader_source, optimizer, scheduler, device=device, epochs=2)

    # the first layer is fine-tuned
    optimizer = optim.Adam(tune_net.layer2.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
    acc = train_customImageNet(tune_net, train_loader_target, val_loader_target, optimizer, scheduler, device=device, epochs=15)
    
    # adding results to dataframe
    dict = {}
    dict['accuracy'] = acc
    dict['lr'] = lr
    dict['state'] = "second layer"  # change when run
    dict['n_subset'] = n_subset
    df_temp = pd.DataFrame(dict, index=[0])
    df = pd.concat([df, df_temp])

# change when run
df.to_csv("Entity30_layer2.csv")

Building model...
Epoch 0, loss = 6.0309, lr 0.00050000
Got 14 / 6000 correct (0.23)

Epoch 1, loss = 4.9546, lr 0.00050000
Got 101 / 6000 correct (1.68)

Epoch 2, loss = 3.9466, lr 0.00050000
Got 263 / 6000 correct (4.38)

Epoch 0, loss = 1.1658, lr 0.00050000
Got 4060 / 6000 correct (67.67)

Epoch 1, loss = 1.2898, lr 0.00050000
Got 4355 / 6000 correct (72.58)

Epoch 0, loss = 2.4440, lr 0.00050000
Got 3306 / 6000 correct (55.10)

Epoch 1, loss = 2.1714, lr 0.00050000
Got 3359 / 6000 correct (55.98)

Epoch 2, loss = 1.5603, lr 0.00050000
Got 3393 / 6000 correct (56.55)

Epoch 3, loss = 1.6110, lr 0.00050000
Got 3416 / 6000 correct (56.93)

Epoch 4, loss = 1.1005, lr 0.00050000
Got 3423 / 6000 correct (57.05)

Epoch 5, loss = 1.3873, lr 0.00050000
Got 3450 / 6000 correct (57.50)

Epoch 6, loss = 1.5951, lr 0.00050000
Got 3473 / 6000 correct (57.88)

Epoch 7, loss = 1.4815, lr 0.00050000
Got 3516 / 6000 correct (58.60)

Epoch 8, loss = 1.2456, lr 0.00050000
Got 3505 / 6000 correct (58.

In [5]:
df = pd.DataFrame(columns=['accuracy', "lr", "state", "n_subset"])

for lr in lrs:
    device = torch.device("mps")
    tune_net = build_resnet50(device)

    optimizer = optim.Adam(tune_net.fc.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
    acc = train_customImageNet(tune_net, train_loader_source, val_loader_source, optimizer, scheduler, device=device, epochs=3)
    
    optimizer = optim.Adam(tune_net.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
    acc = train_customImageNet(tune_net, train_loader_source, val_loader_source, optimizer, scheduler, device=device, epochs=2)

    # the first layer is fine-tuned
    optimizer = optim.Adam(tune_net.layer3.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
    acc = train_customImageNet(tune_net, train_loader_target, val_loader_target, optimizer, scheduler, device=device, epochs=15)
    
    # adding results to dataframe
    dict = {}
    dict['accuracy'] = acc
    dict['lr'] = lr
    dict['state'] = "third layer"  # change when run
    dict['n_subset'] = n_subset
    df_temp = pd.DataFrame(dict, index=[0])
    df = pd.concat([df, df_temp])

# change when run
df.to_csv("Entity30_layer3.csv")

Building model...
Epoch 0, loss = 5.9666, lr 0.00050000
Got 9 / 6000 correct (0.15)

Epoch 1, loss = 5.1181, lr 0.00050000
Got 92 / 6000 correct (1.53)

Epoch 2, loss = 3.7201, lr 0.00050000
Got 268 / 6000 correct (4.47)

Epoch 0, loss = 1.3994, lr 0.00050000
Got 3977 / 6000 correct (66.28)

Epoch 1, loss = 0.6124, lr 0.00050000
Got 4325 / 6000 correct (72.08)

Epoch 0, loss = 1.9072, lr 0.00050000
Got 3670 / 6000 correct (61.17)

Epoch 1, loss = 1.2010, lr 0.00050000
Got 3911 / 6000 correct (65.18)

Epoch 2, loss = 1.5531, lr 0.00050000
Got 3985 / 6000 correct (66.42)

Epoch 3, loss = 1.0230, lr 0.00050000
Got 4123 / 6000 correct (68.72)

Epoch 4, loss = 0.8459, lr 0.00050000
Got 4111 / 6000 correct (68.52)

Epoch 5, loss = 0.4982, lr 0.00050000
Got 4089 / 6000 correct (68.15)

Epoch 6, loss = 0.4236, lr 0.00050000
Got 4124 / 6000 correct (68.73)

Epoch 7, loss = 1.0280, lr 0.00050000
Got 4149 / 6000 correct (69.15)

Epoch 8, loss = 0.9491, lr 0.00050000
Got 4175 / 6000 correct (69.58

In [6]:
df = pd.DataFrame(columns=['accuracy', "lr", "state", "n_subset"])

for lr in lrs:
    device = torch.device("mps")
    tune_net = build_resnet50(device)

    optimizer = optim.Adam(tune_net.fc.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
    acc = train_customImageNet(tune_net, train_loader_source, val_loader_source, optimizer, scheduler, device=device, epochs=3)
    
    optimizer = optim.Adam(tune_net.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
    acc = train_customImageNet(tune_net, train_loader_source, val_loader_source, optimizer, scheduler, device=device, epochs=2)

    # the first layer is fine-tuned
    optimizer = optim.Adam(tune_net.layer4.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
    acc = train_customImageNet(tune_net, train_loader_target, val_loader_target, optimizer, scheduler, device=device, epochs=15)
    
    # adding results to dataframe
    dict = {}
    dict['accuracy'] = acc
    dict['lr'] = lr
    dict['state'] = "fourth layer"  # change when run
    dict['n_subset'] = n_subset
    df_temp = pd.DataFrame(dict, index=[0])
    df = pd.concat([df, df_temp])

# change when run
df.to_csv("Entity30_layer4.csv")

Building model...
Epoch 0, loss = 5.5179, lr 0.00050000
Got 10 / 6000 correct (0.17)

Epoch 1, loss = 5.1437, lr 0.00050000
Got 92 / 6000 correct (1.53)

Epoch 2, loss = 4.4548, lr 0.00050000
Got 261 / 6000 correct (4.35)

Epoch 0, loss = 1.3534, lr 0.00050000
Got 4096 / 6000 correct (68.27)

Epoch 1, loss = 1.0719, lr 0.00050000
Got 4417 / 6000 correct (73.62)

Epoch 0, loss = 1.7378, lr 0.00050000
Got 4610 / 6000 correct (76.83)

Epoch 1, loss = 0.7004, lr 0.00050000
Got 4829 / 6000 correct (80.48)

Epoch 2, loss = 0.8718, lr 0.00050000
Got 4915 / 6000 correct (81.92)

Epoch 3, loss = 1.0646, lr 0.00050000
Got 4916 / 6000 correct (81.93)

Epoch 4, loss = 0.3236, lr 0.00050000
Got 4846 / 6000 correct (80.77)

Epoch 5, loss = 0.5048, lr 0.00050000
Got 4865 / 6000 correct (81.08)

Epoch 6, loss = 0.5932, lr 0.00050000
Got 4924 / 6000 correct (82.07)

Epoch 7, loss = 0.3746, lr 0.00050000
Got 4894 / 6000 correct (81.57)

Epoch 8, loss = 0.2980, lr 0.00050000
Got 4767 / 6000 correct (79.4

In [7]:
df = pd.DataFrame(columns=['accuracy', "lr", "state", "n_subset"])

for lr in lrs:
    device = torch.device("mps")
    tune_net = build_resnet50(device)

    optimizer = optim.Adam(tune_net.fc.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
    acc = train_customImageNet(tune_net, train_loader_source, val_loader_source, optimizer, scheduler, device=device, epochs=3)
    
    optimizer = optim.Adam(tune_net.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
    acc = train_customImageNet(tune_net, train_loader_source, val_loader_source, optimizer, scheduler, device=device, epochs=2)

    # the first layer is fine-tuned
    optimizer = optim.Adam(tune_net.fc.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
    acc = train_customImageNet(tune_net, train_loader_target, val_loader_target, optimizer, scheduler, device=device, epochs=15)
    
    # adding results to dataframe
    dict = {}
    dict['accuracy'] = acc
    dict['lr'] = lr
    dict['state'] = "last layer"  # change when run
    dict['n_subset'] = n_subset
    df_temp = pd.DataFrame(dict, index=[0])
    df = pd.concat([df, df_temp])

# change when run
df.to_csv("Entity30_last_layer.csv")

Building model...
Epoch 0, loss = 5.9945, lr 0.00050000
Got 9 / 6000 correct (0.15)

Epoch 1, loss = 4.5723, lr 0.00050000
Got 103 / 6000 correct (1.72)

Epoch 2, loss = 3.8619, lr 0.00050000
Got 281 / 6000 correct (4.68)

Epoch 0, loss = 1.5909, lr 0.00050000
Got 3940 / 6000 correct (65.67)

Epoch 1, loss = 0.8729, lr 0.00050000
Got 4406 / 6000 correct (73.43)

Epoch 0, loss = 1.4605, lr 0.00050000
Got 3603 / 6000 correct (60.05)

Epoch 1, loss = 1.4955, lr 0.00050000
Got 3836 / 6000 correct (63.93)

Epoch 2, loss = 1.4853, lr 0.00050000
Got 3984 / 6000 correct (66.40)

Epoch 3, loss = 1.3931, lr 0.00050000
Got 4119 / 6000 correct (68.65)

Epoch 4, loss = 1.3521, lr 0.00050000
Got 4198 / 6000 correct (69.97)

Epoch 5, loss = 1.4644, lr 0.00050000
Got 4284 / 6000 correct (71.40)

Epoch 6, loss = 1.4702, lr 0.00050000
Got 4332 / 6000 correct (72.20)

Epoch 7, loss = 0.8255, lr 0.00050000
Got 4369 / 6000 correct (72.82)

Epoch 8, loss = 0.8383, lr 0.00050000
Got 4419 / 6000 correct (73.6

In [4]:
df = pd.DataFrame(columns=['accuracy', "lr", "state", "n_subset"])

for lr in lrs:
    device = torch.device("mps")
    tune_net = build_resnet50(device)

    optimizer = optim.Adam(tune_net.fc.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
    acc = train_customImageNet(tune_net, train_loader_source, val_loader_source, optimizer, scheduler, device=device, epochs=3)
    
    optimizer = optim.Adam(tune_net.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
    acc = train_customImageNet(tune_net, train_loader_source, val_loader_source, optimizer, scheduler, device=device, epochs=2)

    # the first layer is fine-tuned
    optimizer = optim.Adam(tune_net.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
    acc = train_customImageNet(tune_net, train_loader_target, val_loader_target, optimizer, scheduler, device=device, epochs=15)
    
    # adding results to dataframe
    dict = {}
    dict['accuracy'] = acc
    dict['lr'] = lr
    dict['state'] = "all layers"  # change when run
    dict['n_subset'] = n_subset
    df_temp = pd.DataFrame(dict, index=[0])
    df = pd.concat([df, df_temp])

# change when run
df.to_csv("Entity30_all_layers.csv")

Building model...
Epoch 0, loss = 6.2824, lr 0.00050000
Got 12 / 6000 correct (0.20)

Epoch 1, loss = 4.9572, lr 0.00050000
Got 95 / 6000 correct (1.58)

Epoch 2, loss = 4.1311, lr 0.00050000
Got 300 / 6000 correct (5.00)

Epoch 0, loss = 1.8744, lr 0.00050000
Got 3912 / 6000 correct (65.20)

Epoch 1, loss = 1.4899, lr 0.00050000
Got 4419 / 6000 correct (73.65)

Epoch 0, loss = 1.1272, lr 0.00050000
Got 4038 / 6000 correct (67.30)

Epoch 1, loss = 1.1032, lr 0.00050000
Got 4395 / 6000 correct (73.25)

Epoch 2, loss = 0.9017, lr 0.00050000
Got 4431 / 6000 correct (73.85)

Epoch 3, loss = 0.6618, lr 0.00050000
Got 4501 / 6000 correct (75.02)

Epoch 4, loss = 1.1169, lr 0.00050000
Got 4439 / 6000 correct (73.98)

Epoch 5, loss = 0.3912, lr 0.00050000
Got 4419 / 6000 correct (73.65)

Epoch 6, loss = 0.5065, lr 0.00050000
Got 4277 / 6000 correct (71.28)

Epoch 7, loss = 0.6381, lr 0.00050000
Got 4386 / 6000 correct (73.10)

Epoch 8, loss = 0.5735, lr 0.00050000
Got 4286 / 6000 correct (71.4